In [45]:
# coding: utf-8
import pandas as pd
from numpy.random import shuffle
from sklearn import svm, metrics
import pickle


data = pd.read_csv('data9/moment.csv', encoding='gbk')
data = data.values
shuffle(data)
# 数据抽样
train = data[:int(0.8*len(data)), :]  # 选取前80%为训练数据
test = data[int(0.8*len(data)):, :]  # 选取前20%为测试数据


# 构建支持向量机模型
x_train = train[:, 2:]*30
y_train = train[:, 0].astype(int)
x_test = test[:, 2:]*30
y_test = test[:, 0].astype(int)


model = svm.SVC()
model.fit(x_train, y_train)
pickle.dump(model, open('svm.model', 'wb'))
# model = pickle.load('svm.model', 'rb')   # 可以通过该句重新加载svm模型

cm_train = metrics.confusion_matrix(y_train, model.predict(x_train))  # 训练样本的混淆矩阵
cm_test = metrics.confusion_matrix(y_test, model.predict(x_test))  # 测试样本的混淆矩阵

pd.DataFrame(cm_train, index=range(1, 6), columns=range(1, 6)).to_excel('svm_train.xls')
pd.DataFrame(cm_test, index=range(1, 6), columns=range(1, 6)).to_excel('svm_test.xls')
print(model.score(x_test, y_test))

cm_train = metrics.confusion_matrix(y_train, model.predict(x_train)) 
cm_test = metrics.confusion_matrix(y_test, model.predict(x_test)) 


0.9512195121951219


In [50]:
# coding: utf-8
import pandas as pd


data = pd.read_excel('data10/water_heater.xls')
data[u'发生时间'] = pd.to_datetime(data[u'发生时间'], format='%Y%m%d%H%M%S')
data = data[data[u'水流量'] > 0]
threshold = pd.Timedelta(minutes=4)  # 阀值为4分钟
d = data[u'发生时间'].diff() > threshold  # 相邻时间做分差，比较是否大于阀值
data[u'事件编号'] = d.cumsum() + 1  # 通过累计求和的方式进行编号


                     发生时间 开关机状态 加热中 保温中  实际温度  热水量  水流量 加热剩余时间 当前设置温度  事件编号
2     2014-10-19 07:01:56     关   关   关  30°C   0%    8    0分钟   50°C     1
56    2014-10-19 07:38:16     关   关   关  30°C   0%    8    0分钟   50°C     2
381   2014-10-19 09:46:38     关   关   关  29°C   0%   16    0分钟   50°C     3
382   2014-10-19 09:46:40     关   关   关  29°C   0%   13    0分钟   50°C     3
384   2014-10-19 09:47:15     关   关   关  29°C   0%   20    0分钟   50°C     3
404   2014-10-19 11:50:17     关   关   关  29°C   0%   22    0分钟   50°C     4
407   2014-10-19 13:56:21     关   关   关  29°C   0%    8    0分钟   50°C     5
410   2014-10-19 15:34:39     关   关   关  29°C   0%   36    0分钟   50°C     6
411   2014-10-19 15:34:40     关   关   关  29°C   0%   32    0分钟   50°C     6
412   2014-10-19 15:34:45     关   关   关  29°C   0%   31    0分钟   50°C     6
413   2014-10-19 15:34:47     关   关   关  29°C   0%   32    0分钟   50°C     6
414   2014-10-19 15:34:49     关   关   关  29°C   0%   31    0分钟   50°C     6
415   2014-1

In [75]:
# conding: utf-8
import pandas as pd
import numpy as np


data = pd.read_excel('data10/water_heater.xls')
data[u'发生时间'] = pd.to_datetime(data[u'发生时间'], format='%Y%m%d%H%M%S')
data = data[data[u'水流量'] > 0]
threshold = pd.Timedelta(minutes=5)  # 专家阀值


def event_num(ts):
    d = data[u'发生时间'].diff() > ts
    return d.sum() + 1


n = 4  # 使用以后4个点的平均斜率
dt = [pd.Timedelta(minutes=i) for i in np.arange(1, 9, 0.25)]
h = pd.DataFrame(dt, columns=[u'阀值'])
h[u'事件数'] = h[u'阀值'].apply(event_num)  # 计算每个阀值对应的事件数
h[u'斜率'] = h[u'事件数'].diff()/0.25   # 计算每两个相邻点的斜率
h[u'斜率指标'] = h[u'斜率'].abs().rolling(n).mean()  # 采用后n个点的斜率绝对值平均作为斜率指标
ts = h[u'阀值'][h[u'斜率指标'].idxmin() - n]

if ts > threshold:
    ts = pd.Timedelta(minutes=4)
print(ts)

0 days 00:04:00
0 days 00:04:00


In [82]:
# coding: utf-8
"""训练多层神经网络"""
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout


data_train = pd.read_excel('data10/train_neural_network_data.xls')  # 训练样本集
data_test = pd.read_excel('data10/test_neural_network_data.xls')

y_train = data_train.iloc[:, 4].values  # 训练样本标签列
x_train = data_train.iloc[:, 5:17].values  # 训练样本特征
y_test = data_test.iloc[:, 4].values
x_test = data_test.iloc[:, 5:17].values

model = Sequential()
model.add(Dense(17, input_shape=(11, )))
model.add(Activation('relu'))
model.add(Dense(10, input_shape=(17, )))
model.add(Activation('relu'))
model.add(Dense(1, input_shape=(10, )))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(x_train, y_train, epochs=100, batch_size=1)
model.save_weights('net.model')

r = pd.DataFrame(model.predict_classes(x_test), columns=[u'预测结果'])
pd.concat([data_test.iloc[:, :5], r], axis=1).to_excel('day8.xls')
model.predict(x_test)
print(model.predict(x_test))

Epoch 1/100


 1/28 [>.............................] - ETA: 9s - loss: 1.1921e-07

28/28 [==============================] - 0s 14ms/step - loss: 3.4162


Epoch 2/100
 1/28 [>.............................] - ETA: 0s - loss: 1.1921e-07

28/28 [==============================] - 0s 2ms/step - loss: 3.4162


...

Epoch 99/100
 1/28 [>.............................] - ETA: 0s - loss: 1.1921e-07

28/28 [==============================] - 0s 1ms/step - loss: 3.4162


Epoch 100/100
 1/28 [>.............................] - ETA: 0s - loss: 1.1921e-07

28/28 [==============================] - 0s 1ms/step - loss: 3.4162


[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
